In [6]:
import cv2
import numpy as np
import math

# Gauss Blur and Canny Edge 
def BlurCanny(image):
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    gray = cv2.GaussianBlur(gray,(3,3),0.5)
    edges = cv2.Canny(gray,50,150,apertureSize =3)
    return edges

# Hough Line Transform
# Find the peak percentage and distance ratio
# Warrning: img must edge map
def PeakPercentage(img):
    lineV = [] #include number of votes
    tempDic = {}
    wt,ht = img.shape
    try:
        for votes in range(1,int(wt*ht)):
            lines = cv2.HoughLines(img,1,np.pi/180,votes)
            for ele in lines[0]:
                tempDic[str(ele[0])+','+str(ele[1])] = votes
    except TypeError:
        pass
    for ele in tempDic:
        rho = np.float32(ele.split(',')[0])
        theta = np.float32(ele.split(',')[1])
        vote = tempDic[ele]
        lineV.append([rho,theta,vote])

    # The peak percentage
    HoughPeak = {}
    percentage = {}
    for ZT in lineV:
        if ZT[1] in HoughPeak:
            HoughPeak[ZT[1]] += 1    #the number of Hough peaks in the theta bin
        else:
            HoughPeak[ZT[1]] = 1
    for key in HoughPeak:
        # len(lines[0]) :the total number of Hough peaks in the Hough parameter space
        percentage[key] = HoughPeak[key] *1.0 / len(lineV[0]) 
     
    # distance ratio feature
    thetabinAddition = {}
    distanceRatioDic = {}
    zo2 = {}
    wzo = {}
    w ={}
    # min rho
    minRho = lineV[0][0]
    for ele in lineV:
        if minRho > ele[0]:
            minRho = ele[0]
    for ele in lineV:
        if ele[1] not in zo2:
            zo2[ele[1]] = (ele[0]-minRho) * (ele[0]-minRho)
            wzo[ele[1]] = (ele[0]-minRho) * ele[2]
            w[ele[1]] =  ele[2]
        else:
            zo2[ele[1]] += (ele[0]-minRho) * (ele[0]-minRho)
            wzo[ele[1]] += (ele[0]-minRho) * ele[2]
            w[ele[1]] +=  ele[2]
    for key in HoughPeak:
        distanceRatioDic[key] = math.sqrt(zo2[key])/(wzo[key]/w[key])

    return percentage, distanceRatioDic

In [9]:
# rank function
def s(k,q,m,J):
    Numerator = 0
    d1 = 0
    d2 = 0
    Denominator = 0
    # Numerator
    if m == 1:
        for j in range(0,J-m+1): Numerator += q[j]*k[j+m]
        Numerator += q[J]*k[0]
    elif m ==  J-1:
        for j in range(J-m+1,J+1): Numerator += q[j]*k[j+m-J]
        Numerator += q[0]*k[J]
    else:
        for j in range(0,J-m+1): Numerator += q[j]*k[j+m]
        for j in range(J-m+1,J+1): Numerator += q[j]*k[j+m-J]
    for j in range(0,J):
        d1 += q[j]*q[j]
        d2 += k[j]*k[j]
    Denominator = math.sqrt(d1)*math.sqrt(d2)
    return Numerator / Denominator


def rankingFunction(percentage_q, distanceRatio_q,  database):
# sort query img
    pq = []
    rq = []
    for key in sorted(percentage_q):
        pq.append(percentage_q[key])
        rq.append(distanceRatio_q[key])
    # sort 
    ranking = {}
    for path in database:
        # sort kth database img
        pk = []
        rk = []
        for key in sorted(percentage_q):
            pk.append(database[path]['percentage'][key])
            rk.append(database[path]['DistanceRatio'][key])     
        # solve spk rpk
        plist = []
        rlist = []
        J = 180 - 1  #degree
        for m in range(1,J-1):
            plist.append(s(pk,pq,m,J))
            rlist.append(s(rk,rq,m,J))
        p = max(plist)
        r = max(rlist)
        rank = p + r
        ranking[path] = rank
    return ranking

In [13]:
import cv2
import numpy as np

# query INFO.
path_q = 'img/simitsu1.jpg' #query image
img_q = cv2.imread(path_q)
edges_q = BlurCanny(img_q)
percentage_q, distanceRatio_q = PeakPercentage(edges_q)
# 正規化(0~1)
# redistanceRatio_q = {}
# for ele in distanceRatio_q:
#     redistanceRatio_q[ele] = (distanceRatio_q[ele] - min(distanceRatio_q.values()))/ ( max(distanceRatio_q.values()) - min(distanceRatio_q.values()))

# database INFO. should be from database
# already handle
database = ['simitsu1.jpg', 'simitsu4.jpg', 'simitsu5.jpg', 'simitsu6.jpg','TokyoTower001.jpg','TokyoTower002.jpg','TokyoTower003.jpg','TokyoTower004.jpg']
database_t = {}
for path in database:
    tempDic = {}
    img_k = cv2.imread('img/'+path)
    edges_k = BlurCanny(img_k)
    percentage_k, distanceRatio_k = PeakPercentage(edges_k)
    tempDic['percentage'] = percentage_k
    tempDic['DistanceRatio'] = distanceRatio_q
    # 正規化(0~1)
#     redistanceRatio_k = {}
#     for ele in distanceRatio_q:
#         redistanceRatio_k[ele] = (distanceRatio_k[ele] - min(distanceRatio_k.values()))/ ( max(distanceRatio_k.values()) - min(distanceRatio_k.values()))
#     tempDic['DistanceRatio'] = redistanceRatio_k
    database_t[path] = tempDic.copy()

rankDic = rankingFunction(percentage_q, distanceRatio_q, database_t)
print rankDic

KeyError: 'DistanceRatio'

In [ ]:
for path in database:
    tempDic = {}
    img_k = cv2.imread('img/'+path)
    edges_k = BlurCanny(img_k)
    percentage_k, distanceRatio_k = PeakPercentage(edges_k)
    tempDic['percentage'] = percentage_k
    tempDic['DistanceRatio'] = distanceRatio_q
    database_t[path] = tempDic.copy()
rankDic = rankingFunction(percentage_q, distanceRatio_q, database_t)
print rankDic